In [24]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsRegressor
# from statsmodels.discrete.discrete_model import Logit
# from statsmodels.discrete.discrete_model import LogitResults
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [2]:
data = pd.read_csv('creditcard.csv')

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data_reg = data.copy()
data_reg.drop(['Time'], axis=1, inplace=True)
data_reg['log_amount'] = np.log(list(data_reg['Amount'] + 1))
data_reg.drop('Amount', axis=1, inplace=True)

normal_data = data_reg[data['Class'] == 0]
normal_data.drop(['Class'], axis=1, inplace=True)
fraud_data = data_reg[data['Class'] == 1]
fraud_data.drop(['Class'], axis=1, inplace=True)
# fraud_data

normal_y = normal_data['log_amount']
fraud_y = fraud_data['log_amount']

normal_X = normal_data.drop('log_amount', axis=1)
fraud_X = fraud_data.drop('log_amount', axis=1)


/Users/yijunshen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/yijunshen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
np.random.seed(182)
msk = np.random.rand(len(normal_data)) < 0.75
normal_train = normal_data[msk]
normal_test = normal_data[~msk]
# normal_train.shape, normal_test.shape

normal_X_train = normal_train.drop('log_amount', axis=1)
normal_y_train = normal_train['log_amount']

normal_X_test = normal_test.drop('log_amount', axis=1)
normal_y_test = normal_test['log_amount']

In [6]:
msk = np.random.rand(len(fraud_data)) < 0.75
fraud_train = fraud_data[msk]
fraud_test = fraud_data[~msk]
# fraud_train.shape, fraud_test.shape

fraud_X_train = fraud_train.drop('log_amount', axis=1)
fraud_y_train = fraud_train['log_amount']

fraud_X_test = fraud_test.drop('log_amount', axis=1)
fraud_y_test = fraud_test['log_amount']

## Linear Regression

In [7]:
linear_regression = LinearRegression(fit_intercept=True)
linear_regression.fit(normal_X_train, normal_y_train)
normal_train_score = linear_regression.score(normal_X_train, normal_y_train)
normal_test_score = linear_regression.score(normal_X_test, normal_y_test)
print("Normal Transaction linear regression train score:", normal_train_score)
print("Normal Transaction linear regression test score:", normal_test_score)

Normal Transaction linear regression train score: 0.401224393484
Normal Transaction linear regression test score: 0.356168560254


In [8]:
linear_regression = LinearRegression(fit_intercept=True)
linear_regression.fit(fraud_X_train, fraud_y_train)
fraud_train_score = linear_regression.score(fraud_X_train, fraud_y_train)
fraud_test_score = linear_regression.score(fraud_X_test, fraud_y_test)
print("Fraud Transaction linear regression train score:", fraud_train_score)
print("Fraud Transaction linear regression test score:", fraud_test_score)

Fraud Transaction linear regression train score: 0.640395208557
Fraud Transaction linear regression test score: 0.518157716409


## Ridge Regression

In [9]:
lambdas = [.001,.005,1,5,10,50,100,500,1000]

In [10]:
ridge_regression = RidgeCV(alphas=lambdas, fit_intercept=True)
ridge_regression.fit(normal_X_train, normal_y_train)

print("Normal Transaction Ridge regression train Score:", ridge_regression.score(normal_X_train, normal_y_train))
print("Normal Transaction Ridge regression test Score:", ridge_regression.score(normal_X_test, normal_y_test))

# best_alpha_ridge = ridge_regression.alpha_
# best_alpha_ridge

Normal Transaction Ridge regression train Score: 0.401218788257
Normal Transaction Ridge regression test Score: 0.356274466081


In [11]:
ridge_regression = RidgeCV(alphas=lambdas, fit_intercept=True)
ridge_regression.fit(fraud_X_train, fraud_y_train)

print("Fraud Transaction Ridge regression train Score:", ridge_regression.score(fraud_X_train, fraud_y_train))
print("Fraud Transaction Ridge regression test Score:", ridge_regression.score(fraud_X_test, fraud_y_test))

Fraud Transaction Ridge regression train Score: 0.635101029146
Fraud Transaction Ridge regression test Score: 0.528602962983


## Lasso Regression

In [12]:
lasso_regression = LassoCV(alphas=lambdas, fit_intercept=True)
lasso_regression.fit(normal_X_train, normal_y_train)

print("Normal Transaction Lasso regression train Score:", lasso_regression.score(normal_X_train, normal_y_train))
print("Normal Transaction Lasso regression test Score:", lasso_regression.score(normal_X_test, normal_y_test))

Normal Transaction Lasso regression train Score: 0.400788465934
Normal Transaction Lasso regression test Score: 0.356749565618


In [13]:
lasso_regression = LassoCV(alphas=lambdas, fit_intercept=True)
lasso_regression.fit(fraud_X_train, fraud_y_train)

print("Fraud Transaction Lasso regression train Score:", lasso_regression.score(fraud_X_train, fraud_y_train))
print("Fraud Transaction Lasso regression test Score:", lasso_regression.score(fraud_X_test, fraud_y_test))

Fraud Transaction Lasso regression train Score: 0.639963404558
Fraud Transaction Lasso regression test Score: 0.520325320109


## Add polynomial and interation terms

In [14]:
normal_poly = normal_data.copy()
fraud_poly = fraud_data.copy()

columns = list(normal_poly.columns)
predictors = columns[:-1]

for predictor in predictors:
    for i in range(1,3):
        normal_poly[predictor + '_' + str(i)] = normal_poly[predictor]**i
        fraud_poly[predictor + '_' + str(i)] = fraud_poly[predictor]**i

# normal_poly.columns, fraud_poly.columns

In [15]:
msk = np.random.rand(len(normal_poly)) < 0.75
normal_poly_train = normal_poly[msk]
normal_poly_test = normal_poly[~msk]

normal_poly_X_train = normal_poly_train.drop('log_amount', axis=1)
normal_poly_y_train = normal_poly_train['log_amount']

normal_poly_X_test = normal_poly_test.drop('log_amount', axis=1)
normal_poly_y_test = normal_poly_test['log_amount']

In [16]:
msk = np.random.rand(len(fraud_poly)) < 0.75
fraud_poly_train = fraud_poly[msk]
fraud_poly_test = fraud_poly[~msk]

fraud_poly_X_train = fraud_poly_train.drop('log_amount', axis=1)
fraud_poly_y_train = fraud_poly_train['log_amount']

fraud_poly_X_test = fraud_poly_test.drop('log_amount', axis=1)
fraud_poly_y_test = fraud_poly_test['log_amount']

In [17]:
linear_regression = LinearRegression(fit_intercept=True)
linear_regression.fit(normal_poly_X_train, normal_poly_y_train)
normal_poly_train_score = linear_regression.score(normal_poly_X_train, normal_poly_y_train)
normal_poly_test_score = linear_regression.score(normal_poly_X_test, normal_poly_y_test)
print("Normal Transaction linear regression with polynomial terms train score:", normal_poly_train_score)
print("Normal Transaction linear regression with polynomial terms test score:", normal_poly_test_score)

Normal Transaction linear regression with polynomial terms train score: 0.456739179419
Normal Transaction linear regression with polynomial terms test score: 0.462759367049


In [18]:
linear_regression = LinearRegression(fit_intercept=True)
linear_regression.fit(fraud_poly_X_train, fraud_poly_y_train)
fraud_poly_train_score = linear_regression.score(fraud_poly_X_train, fraud_poly_y_train)
fraud_poly_test_score = linear_regression.score(fraud_poly_X_test, fraud_poly_y_test)
print("Fraud Transaction linear regression with polynomial terms train score:", fraud_poly_train_score)
print("Fraud Transaction linear regression with polynomial terms test score:", fraud_poly_test_score)

Fraud Transaction linear regression with polynomial terms train score: 0.715601713841
Fraud Transaction linear regression with polynomial terms test score: 0.500711518748


## Ridge Regression for Linear Regression with Polynomial Terms

In [19]:
ridge_regression = RidgeCV(alphas=lambdas, fit_intercept=True)
ridge_regression.fit(normal_poly_X_train, normal_poly_y_train)

print("Normal Transaction Ridge regression with polynomial terms train score:", \
      ridge_regression.score(normal_poly_X_train, normal_poly_y_train))
print("Normal Transaction Ridge regression with polynomial terms test score:", \
      ridge_regression.score(normal_poly_X_test, normal_poly_y_test))

Normal Transaction Ridge regression with polynomial terms train score: 0.456728831367
Normal Transaction Ridge regression with polynomial terms test score: 0.462717173585


In [20]:
ridge_regression = RidgeCV(alphas=lambdas, fit_intercept=True)
ridge_regression.fit(fraud_poly_X_train, fraud_poly_y_train)

print("Fraud Transaction Ridge regression with polynomial terms train score:", \
      ridge_regression.score(fraud_poly_X_train, fraud_poly_y_train))
print("Fraud Transaction Ridge regression with polynomial terms test score:", \
      ridge_regression.score(fraud_poly_X_test, fraud_poly_y_test))

Fraud Transaction Ridge regression with polynomial terms train score: 0.706706955657
Fraud Transaction Ridge regression with polynomial terms test score: 0.554782908376


## Lasso Regression for Linear Regression with Polynomial Terms

In [21]:
lasso_regression = LassoCV(alphas=lambdas, fit_intercept=True)
lasso_regression.fit(normal_poly_X_train, normal_poly_y_train)

print("Normal Transaction Lasso regression with polynomial terms train score:", \
      lasso_regression.score(normal_poly_X_train, normal_poly_y_train))
print("Normal Transaction Lasso regression with polynomial terms test score:", \
      lasso_regression.score(normal_poly_X_test, normal_poly_y_test))

Normal Transaction Lasso regression with polynomial terms train score: 0.455870341706
Normal Transaction Lasso regression with polynomial terms test score: 0.46201646971


In [23]:
lasso_regression = LassoCV(alphas=lambdas, fit_intercept=True)
lasso_regression.fit(fraud_poly_X_train, fraud_poly_y_train)

print("Fraud Transaction Lasso regression with polynomial terms train score:", \
      lasso_regression.score(fraud_poly_X_train, fraud_poly_y_train))
print("Fraud Transaction Lasso regression with polynomial terms test score:", \
      lasso_regression.score(fraud_poly_X_test, fraud_poly_y_test))

Fraud Transaction Lasso regression with polynomial terms train score: 0.389091650334
Fraud Transaction Lasso regression with polynomial terms test score: 0.325922153464


/Users/yijunshen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## k-Nearest Neighbors

In [25]:
knn_model = KNeighborsRegressor(n_neighbors = 10)
knn_model.fit(normal_X_train, normal_y_train)
knn_model.score(normal_X_train, normal_y_train)

0.73523463621825036

In [ ]:
knn_model.score(normal_X_test, normal_y_test)